# TensorFlow Dataset API

Lab from https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/solutions/2_dataset_api.ipynb with personal notes nd comments.

**Learning Objectives**
1. Learn how to use tf.data to read data from memory
1. Learn how to use tf.data in a training loop
1. Learn how to use tf.data to read data from disk
1. Learn how to write production input pipelines with feature engineering (batching, shuffling, etc.)


In this notebook, we will start by refactoring the linear regression we implemented in the previous lab so that it takes data from a`tf.data.Dataset`, and we will learn how to implement **stochastic gradient descent** with it. In this case, the original dataset will be synthetic and read by the `tf.data` API directly  from memory.

In a second part, we will learn how to load a dataset with the `tf.data` API when the dataset resides on disk.

Each learning objective will correspond to a __#TODO__  in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/2_dataset_api.ipynb).


In [3]:
import os
from pprint import pprint
from pathlib import Path
import pandas as pd
import warnings

import tensorflow as tf
print(tf.version.VERSION)

if tf.version.VERSION != '2.19.0':
    warnings.warn('Running an untested version of the Tensorflow API.')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2.19.0


## Loading data from memory

### Creating the dataset

Let's consider the synthetic dataset of the previous section:

In [2]:
N_POINTS = 10
X = tf.constant(range(N_POINTS), dtype=tf.float32)
Y = 2 * X + 10

We begin with implementing a function that takes as input

- our $X$ and $Y$ vectors of synthetic data generated by the linear function $y= 2x + 10$
- the number of passes over the dataset we want to train on (`epochs`)
- the size of the batches the dataset (`batch_size`)

and returns a `tf.data.Dataset`: 

In [ ]:
def create_dataset(X, Y, epochs, batch_size, is_training = False):
    """Builds a dataset with rules on how to iterate through it (during training).
    """

    # Note: # the input can also be passed as: {'X': X, 'Y': Y}
    dataset = tf.data.Dataset.from_tensor_slices( (X, Y)) 

    # Note 1: The last batch may not contain the exact number of elements, hence drop_remainder=True.
    # Note 2: epochs sets a maximum to the number of iterations in the training process. If epochs 
    #   is null, we iterate through the dataset infinitely. 
    
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)

    if is_training:
        # If for trainig 
        dataset = dataset.shuffle(buffer_size=10, seed=12345)

    # Most dataset input pipelines should end with a call to `prefetch`. This
    # allows later elements to be prepared while the current element is being
    # processed. This often improves latency and throughput, at the cost of
    # using additional memory to store prefetched elements.

    # Note: Like other `Dataset` methods, prefetch operates on the
    # elements of the input dataset. It has no concept of examples vs. batches.
    # `examples.prefetch(2)` will prefetch two elements (2 examples),
    # while `examples.batch(20).prefetch(2)` will prefetch 2 elements
    # (2 batches, of 20 examples each).
    dataset.prefetch(buffer_size=1)

    return dataset

Let's test our function by iterating twice over our dataset in batches of 3 datapoints:

In [4]:
BATCH_SIZE = 3
EPOCHS = 2
dataset = create_dataset(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE)

for i, (x, y) in enumerate(dataset):
    print("x:", x.numpy(), "y:", y.numpy())
    assert len(x) == BATCH_SIZE
    assert len(y) == BATCH_SIZE

x: [0. 1. 2.] y: [10. 12. 14.]
x: [3. 4. 5.] y: [16. 18. 20.]
x: [6. 7. 8.] y: [22. 24. 26.]
x: [9. 0. 1.] y: [28. 10. 12.]
x: [2. 3. 4.] y: [14. 16. 18.]
x: [5. 6. 7.] y: [20. 22. 24.]


### Loss function and gradients

The loss function and the function that computes the gradients are the same as before:

In [10]:
def loss_mse(X, Y, w0, w1):
    # Note: we write this as if we are deailing wiht one instance at the time 
    # (hence, X is a 1D vector, or a scalar in this example)
    Y_hat = w0 * X + w1
    errors = (Y_hat - Y)**2
    return tf.reduce_mean(errors)

def compute_gradients(X, Y, w0, w1):
    with tf.GradientTape() as tape:
        loss = loss_mse(X, Y, w0, w1)
    return tape.gradient(loss, [w0, w1])

### (Manual) Training loop

In the traning loop, we will iterate directly on the `tf.data.Dataset` generated by our `create_dataset` function. 

In [ ]:
EPOCHS = 250
BATCH_SIZE = 2
LEARNING_RATE = .02

MSG = "STEP {step} - loss: {loss}, w0: {w0}, w1: {w1}\n"

w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)

dataset = create_dataset(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE)

for step, (X_batch, Y_batch) in enumerate(dataset): 
    # compute delta
    dw0, dw1 = compute_gradients(X_batch, Y_batch, w0, w1)
    w0.assign_sub(dw0 * LEARNING_RATE)
    w1.assign_sub(dw1 * LEARNING_RATE)
    
    # print(MSG.format(step=step, loss=loss, w0=w0.numpy(), w1=w1.numpy()))
    
    if step % 100 == 0:
        loss = loss_mse(X, Y, w0, w1)
        print(MSG.format(step=step, loss=loss, w0=w0.numpy(), w1=w1.numpy()))
        
assert loss < 0.0001
assert abs(w0 - 2) < 0.001
assert abs(w1 - 10) < 0.001

STEP 0 - loss: 331.1055908203125, w0: 0.23999999463558197, w1: 0.4399999976158142

STEP 100 - loss: 3.2297768592834473, w0: 2.55655837059021, w1: 6.674341678619385

STEP 200 - loss: 0.48055925965309143, w0: 2.2146825790405273, w1: 8.717182159423828

STEP 300 - loss: 0.07150308042764664, w0: 2.082810878753662, w1: 9.505172729492188

STEP 400 - loss: 0.010638890787959099, w0: 2.03194260597229, w1: 9.809128761291504

STEP 500 - loss: 0.001582985743880272, w0: 2.012321710586548, w1: 9.926374435424805

STEP 600 - loss: 0.00023550672631245106, w0: 2.0047526359558105, w1: 9.971602439880371

STEP 700 - loss: 3.50688278558664e-05, w0: 2.0018346309661865, w1: 9.989042282104492

STEP 800 - loss: 5.220536877459381e-06, w0: 2.000706911087036, w1: 9.995771408081055

STEP 900 - loss: 7.789132610014349e-07, w0: 2.0002737045288086, w1: 9.998367309570312

STEP 1000 - loss: 1.1540369371232373e-07, w0: 2.000105381011963, w1: 9.999371528625488

STEP 1100 - loss: 1.7078491509892046e-08, w0: 2.00004005432128

## Loading data from disk

### Use tf.data to read the CSV files

The `tf.data` API can easily read csv files using the helper function tf.data.experimental.make_csv_dataset

If you have TFRecords (which is recommended), you may use tf.data.experimental.make_batched_features_dataset

The first step is to define 

- the feature names into a list `CSV_COLUMNS`
- their default values into a list `DEFAULTS`

In [40]:
PATH_TO_DATA_FOLDER = Path('data')

# The data have no headers: define the feature names here
CSV_COLUMNS = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'key'
]
# Target variable
LABEL_COLUMN = 'fare_amount'

# Defining the default values into a list `DEFAULTS`
DEFAULTS = [[0.0], ['na'], [0.0], [0.0], [0.0], [0.0], [0.0], ['na']]

df = pd.read_csv( 'data/taxi-train.csv', names = CSV_COLUMNS, nrows=10)
df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,key
0,11.3,2011-01-28 20:42:59 UTC,-73.999022,40.739146,-73.990369,40.717866,1,0
1,7.7,2011-06-27 04:28:06 UTC,-73.987443,40.729221,-73.979013,40.758641,1,1
2,10.5,2011-04-03 00:54:53 UTC,-73.982539,40.735725,-73.954797,40.778388,1,2
3,16.2,2009-04-10 04:11:56 UTC,-74.001945,40.740505,-73.913850,40.758559,1,3
4,33.5,2014-02-24 18:22:00 UTC,-73.993372,40.753382,-73.860900,40.732897,2,4


In [ ]:
# Create a tensorflow dataset for exploration only (no transformations)
def create_dataset_without_transformations(pattern: str, batch_size=10):
    """Load csv data from a path (which can be a pattern, i.e. include the `*` symbol) and return a
    tf.data.Dataset.
    """

    return tf.data.experimental.make_csv_dataset(
        # Note that this is, by default
        pattern,
        batch_size=batch_size,
        column_names=CSV_COLUMNS,
        column_defaults=DEFAULTS)

tempds = create_dataset_without_transformations( 'data/taxi-*.csv', batch_size=3)
print(tempds)

<_PrefetchDataset element_spec=OrderedDict([('fare_amount', TensorSpec(shape=(3,), dtype=tf.float32, name=None)), ('pickup_datetime', TensorSpec(shape=(3,), dtype=tf.string, name=None)), ('pickup_longitude', TensorSpec(shape=(3,), dtype=tf.float32, name=None)), ('pickup_latitude', TensorSpec(shape=(3,), dtype=tf.float32, name=None)), ('dropoff_longitude', TensorSpec(shape=(3,), dtype=tf.float32, name=None)), ('dropoff_latitude', TensorSpec(shape=(3,), dtype=tf.float32, name=None)), ('passenger_count', TensorSpec(shape=(3,), dtype=tf.float32, name=None)), ('key', TensorSpec(shape=(3,), dtype=tf.string, name=None))])>


Note that this is a **prefetched dataset, where each element is an `OrderedDict` whose keys are the feature names and whose values are tensors of shape `(batch_size,)` (i.e. vectors).**

Let's iterate over the two first element of this dataset using `dataset.take(2)` and let's convert them ordinary Python dictionary with numpy array as values for more readability:

In [29]:
for data in tempds.take(2):
    pprint({k: v.numpy() for k, v in data.items()})
    print("\n")

{'dropoff_latitude': array([40.744682, 40.75461 , 40.735806], dtype=float32),
 'dropoff_longitude': array([-73.98476 , -73.9798  , -73.989624], dtype=float32),
 'fare_amount': array([15. ,  4. ,  5.3], dtype=float32),
 'key': array([b'1714', b'236', b'437'], dtype=object),
 'passenger_count': array([1., 1., 1.], dtype=float32),
 'pickup_datetime': array([b'2014-02-24 18:22:00 UTC', b'2014-10-06 15:16:00 UTC',
       b'2009-11-13 07:20:07 UTC'], dtype=object),
 'pickup_latitude': array([40.778023, 40.761875, 40.745148], dtype=float32),
 'pickup_longitude': array([-73.94582 , -73.97502 , -73.998215], dtype=float32)}


{'dropoff_latitude': array([40.758404, 40.787586, 40.7581  ], dtype=float32),
 'dropoff_longitude': array([-73.98778, -73.97701, -73.97735], dtype=float32),
 'fare_amount': array([5.5, 7.5, 6. ], dtype=float32),
 'key': array([b'1904', b'1135', b'57'], dtype=object),
 'passenger_count': array([1., 1., 1.], dtype=float32),
 'pickup_datetime': array([b'2013-05-28 12:58:30 UTC

### Transforming the features

In this step we want to do two things:
1. Do something such that, when we iterate through the dataset for custom training, we can separate features from label (as in previous example).
2. Apply some transformation to the features (e.g. drop some features).

This can be achieved writing a custom function that does both (1) and (2), and pass it to the `tf.data.Dataset` object through the `map` method, that will apply the transformation to each element of the dataset as it iterates. If used with the `prefetch` method, this can lead to a very efficient implementation, where we transform the data while we train the model.



Let's first implement a function that takes as input a row (represented as an `OrderedDict` in our `tf.data.Dataset` as above) and then returns a tuple with two elements:

* The first element being the same `OrderedDict` with the label dropped
* The second element being the label itself (`fare_amount`)

Note that we will need to also remove the `key` and `pickup_datetime` column, which we won't use.

In [23]:
UNWANTED_COLS = ['pickup_datetime', 'key']
from collections import OrderedDict

def features_and_labels(row_data: OrderedDict):
    """This function applies a transformation to each row of the dataset. As we saw that each row is
    treated as an OrderedDict, we need to write the function to operate on it.

    Returns:
        OrderedDict: containing the features we want to use.
        label_value: the value needed for training the model. Why this is not an OrderedDict?
    """



    # if we wanted to preserve row_data, here we would need to make a copy of this dictionary. However, 
    # in the context of batch training and iteration of the dataset losing info from row_data is 
    # irrelevant. We are, in fact, continuously readying new data (i.e. a new row_data object), dropping
    # some keys and moving on to the next one.
    features = row_data
    
    # remove label...
    label_value = features.pop(LABEL_COLUMN)
    
    # ... and unwanted features
    for key_to_remove in UNWANTED_COLS:
        _ = features.pop(key_to_remove)
    
    return features, label_value

Let's iterate over 2 examples from our `tempds` dataset and apply our `feature_and_labels`
function to each of the examples to make sure it's working:

In [30]:
BATCH_SIZE = 3
for row_data in tempds.take(2):
    features, label = features_and_labels(row_data)
    pprint(features)
    print(label, "\n")
    # assert UNWANTED_COLS[0] not in features.keys()
    # assert UNWANTED_COLS[1] not in features.keys()
    assert label.shape == [BATCH_SIZE]

OrderedDict([('pickup_longitude',
              <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-73.99418, -73.96282, -73.9576 ], dtype=float32)>),
             ('pickup_latitude',
              <tf.Tensor: shape=(3,), dtype=float32, numpy=array([40.73548 , 40.76987 , 40.761368], dtype=float32)>),
             ('dropoff_longitude',
              <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-73.986336, -73.9871  , -73.954254], dtype=float32)>),
             ('dropoff_latitude',
              <tf.Tensor: shape=(3,), dtype=float32, numpy=array([40.729626, 40.728333, 40.77478 ], dtype=float32)>),
             ('passenger_count',
              <tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>)])
tf.Tensor([ 5.  13.5  6.3], shape=(3,), dtype=float32) 

OrderedDict([('pickup_longitude',
              <tf.Tensor: shape=(3,), dtype=float32, numpy=array([-73.9828 , -73.98509, -73.96314], dtype=float32)>),
             ('pickup_latitude',
              <t

### Putting things together: reading from disk and and transform while training

In [37]:
# Create a tensorflow dataset for exploration only (no transformations)
def create_dataset_with_transformations(pattern: str, batch_size=10, mode='eval'):
    """Load csv data from a path (which can be a pattern, i.e. include the `*` symbol) and return a
    tf.data.Dataset.
    """

    dataset = tf.data.experimental.make_csv_dataset(
        pattern,
        batch_size=batch_size,
        column_names=CSV_COLUMNS,
        column_defaults=DEFAULTS)

    dataset = dataset.map(features_and_labels).cache()

    if mode == 'train':
        dataset = dataset.shuffle(12345).repeat()

    # take advantage of multi-threading; 1=AUTOTUNE
    # tf.data.AUTOTUNE
    dataset = dataset.prefetch(1)
    
    return dataset

Let's test that our batches are of the right size:

In [50]:
BATCH_SIZE = 3

tempds = create_dataset_with_transformations( 'data/taxi-train*.csv', batch_size=BATCH_SIZE)

for X_batch, Y_batch in tempds.take(2):
    pprint({k: v.numpy() for k, v in X_batch.items()})
    print(f'Y = {list(Y_batch.numpy())}', end="\n\n")
    assert len(Y_batch) == BATCH_SIZE

{'dropoff_latitude': array([40.643425, 40.75328 , 40.75985 ], dtype=float32),
 'dropoff_longitude': array([-73.78996, -73.97293, -73.98227], dtype=float32),
 'passenger_count': array([1., 1., 1.], dtype=float32),
 'pickup_latitude': array([40.761078, 40.758465, 40.732746], dtype=float32),
 'pickup_longitude': array([-73.97889, -73.98506, -73.98142], dtype=float32)}
Y = [np.float32(52.0), np.float32(5.5), np.float32(10.5)]

{'dropoff_latitude': array([40.763275, 40.741783, 40.78058 ], dtype=float32),
 'dropoff_longitude': array([-73.99644, -73.99738, -73.94664], dtype=float32),
 'passenger_count': array([3., 1., 5.], dtype=float32),
 'pickup_latitude': array([40.74281 , 40.751762, 40.75532 ], dtype=float32),
 'pickup_longitude': array([-73.98438 , -73.99364 , -73.983086], dtype=float32)}
Y = [np.float32(8.9), np.float32(4.9), np.float32(12.1)]



**TO SEE HOW TO USE THIS FOR TRAINING, GO TO: https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/solutions/1_training_at_scale_vertex.ipynb**

### Shuffling

When training a deep learning model in batches over multiple workers, it is helpful if we shuffle the data. That way, different workers will be working on different parts of the input file at the same time, and so averaging gradients across workers will help. Also, during training, we will need to read the data indefinitely.

Let's refactor our `create_dataset` function so that it shuffles the data, when the dataset is used for training.

We will introduce an additional argument `mode` to our function to allow the function body to distinguish the case
when it needs to shuffle the data (`mode == 'train'`) from when it shouldn't (`mode == 'eval'`).

Also, before returning we will want to prefetch 1 data point ahead of time (`dataset.prefetch(1)`) to speed-up training:

**Lab Task #4c:** The last step of our `tf.data` dataset will specify shuffling and repeating of our dataset pipeline. Complete the code below to add these three steps to the Dataset pipeline
1. follow the `.map(...)` operation which extracts features and labels with a call to `.cache()` the result.
2. during training, use `.shuffle(...)` and `.repeat()` to shuffle batches and repeat the dataset
3. use `.prefetch(...)` to take advantage of multi-threading and speedup training.

Let's check that our function works well in both modes:

In [ ]:
tempds = create_dataset('toy-data/taxi-train*', 2, 'train')
print(list(tempds.take(1)))

In [ ]:
tempds = create_dataset('toy-data/taxi-train*', 2, 'eval')
print(list(tempds.take(1)))

In the next notebook, we will build the model using this input pipeline.

Copyright 2021 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.